In [1]:
# Import directories
import pandas as pd
import numpy as np
from joblib import dump, load

In [74]:
def classify_blink(df_blink, clf, add_num, levels=5, num_frames=6):
    
    # === For the amount of added frames before the blink detection, ===
    # === add a mirror image to the end of the dataframe             ===
    #df_blink_reverse = df_blink.iloc[:add_num].iloc[::-1]
    #df_blink_extended = pd.concat([df_blink, df_blink_reverse]).reset_index(drop=True)
    df_blink_extended = df_blink.copy()
    
    # === Reduce data: save only strides of four samples ===
    zero_start = 2
    zero_end = len(df_blink_extended.index) - 2
    
    if (zero_end - zero_start > 4):
        df_blink_offset = df_blink_extended.drop(df_blink_extended.index[zero_start + 4 : zero_end + 1]).reset_index(drop=True)
    else:
        df_blink_offset = df_blink_extended.copy()
        
    # === Add features from neighboring frames ===
    features = ['ear','poly']
    
    for offset in range(num_frames*(-1), num_frames+1):
        
        # Do something only if the offset isn't zero
        if(offset == 0):
            continue
        
        for i in range(len(df_blink_offset.index)):
                # Add a column for each selected feature
                for feature in features:
                    new_column = feature + '_' + str(offset)
                    if (i + offset < 0) or (i + offset > len(df_blink_offset.index) - 1):
                        df_blink_offset.loc[i, new_column] = 1
                    else:
                        if np.isnan(df_blink_offset.loc[i + offset, feature]):
                            df_blink_offset.loc[i, new_column]  = 1
                        else:
                            df_blink_offset.loc[i, new_column]  = df_blink_offset.loc[i + offset, feature]
        
        df_blink_offset.loc[:,'palsy_eye'] = np.ones(len(df_blink_offset.index))
        
    # === Create X test vector ===
    # Add the new columns to the feature list
    new_features = []
    
    for feature in features:
        add_cols   = [col for col in df_blink_offset.columns if col.startswith(feature)]
        new_features = new_features + add_cols
    new_features.append('palsy_eye')
    
    Xtest = df_blink_offset[new_features]

    # === Classify the data using the given classifier ===
    y_pred = clf.predict(Xtest)
    df_blink_offset.loc[:,'prediction'] = y_pred / (levels - 1)
    
    print(df_blink_offset[['ear','poly','label','prediction']])
    
    # === Find the minimum predicted label and consider it as 'blink score' ===
    min_pred  = min(df_blink_offset.iloc[1:-1,:]['prediction'])
    min_pred_idx = df_blink_offset.loc[df_blink_offset['prediction'] == min_pred].index
    min_pred_mid = min_pred_idx[min(len(min_pred_idx) - 1, round(len(min_pred_idx)/2))]
    min_locs = df_blink_offset.loc[min_pred_mid-1:min_pred_mid+1,:]['prediction'].reset_index(drop=True)
    
    if len(min_locs) > 2:
            if (min_pred == 0.5 and min_locs[0] == 0.75 and min_locs[2] == 0.75):
                min_pred_final = min_pred
            elif (min_pred == 0 and min_locs[0] == 0.75 and min_locs[2] == 1):
                min_pred_final = min_pred
            elif (min_pred == 0 and min_locs[0] == 0.25 and min_locs[2] == 0.5):
                min_pred_final = min_pred
            elif (min_pred == 0.25 and min_locs[0] == 1 and min_locs[2] == 0.75):
                min_pred_final = min_pred
            else:
                min_pred_final = np.median(min_locs)
    
    elif (min_pred == 0.5 and min_locs[0] == 0.5 and df_blink_offset.iloc[-1,:]['prediction'] == 0.5):
                min_pred_final = 0.75
    else:
        min_pred_final = np.median(min_locs)
    
    return min_pred_final

In [77]:
file_name = 'palsy_18_r_30fps'
palsy_loc = 'right'
log_name = file_name + "_log.txt"
blink_buffer = 4
blink_consec = 2

log_file = open("../5_final_product/files/"+file_name+"/"+log_name, 'r')

i = 1

df_db_xlsx = pd.ExcelFile("files/database.xlsx")
df_db = pd.read_excel(df_db_xlsx,'database')

clf = load('model.joblib')

for line in log_file:
    line_split = line.split(' ')
    blink_range = line_split[3].split(',')
    first_frame = int(blink_range[0][1:]) - (blink_buffer - blink_consec)
    last_frame = int(blink_range[1][:-2]) + (blink_buffer - blink_consec)
    
    print('======= BLINK NUMBER ' + str(i) + ": =======")
    blink_xlsx = pd.ExcelFile("../5_final_product/files/"+file_name+"/"+file_name+"_blink_"+str(i)+".xlsx")
    df_blink = pd.read_excel(blink_xlsx,'palsy_eye')
    
    # Cut the first blink_consec frames
    df_blink = df_blink.drop(df_blink.index[:blink_consec]).reset_index(drop=True)
    
    array = list(range(first_frame, last_frame+1))
    #array = list(range(55,62))
    df_blink.loc[:,'label'] = df_db.loc[(df_db['video_name'] == file_name) & (df_db['eye_location'] == palsy_loc)
                                          & (df_db['frame_number'].isin(array))].reset_index(drop=True).loc[:,'label']
    #df_blink = df_db.loc[(df_db['video_name'] == file_name) & (df_db['eye_location'] == palsy_loc)
    #                                      & (df_db['frame_number'].isin(array))].reset_index(drop=True)[['ear','poly','palsy_eye','label']]
    
    #df_blink = df_blink.drop('palsy', axis=1)
    df_blink.fillna(1, inplace=True)
    #print(df_blink)
    
    print("LABEL: ", min(df_blink.loc[:,'label']))
    print("CLASSIFICATION: ", classify_blink(df_blink, clf, blink_buffer))
    #print (df_db.loc[(df_db['video_name'] == file_name) & (df_db['eye_location'] == palsy_loc)
    #                                      & (df_db['frame_number'].isin(array))].reset_index(drop=True).loc[:,'label'])
    i = i + 1


======= BLINK NUMBER 1: =======
LABEL:  0.25
    ear  poly  label  prediction
0  0.58  0.46   0.75        1.00
1  0.45  0.38   0.75        0.75
2  0.19  0.19   0.50        0.75
3  0.06  0.08   0.25        0.50
4  0.04  0.10   0.25        0.50
5  0.13  0.14   0.25        0.75
6  0.48  0.45   0.75        1.00
CLASSIFICATION:  0.5
======= BLINK NUMBER 2: =======
LABEL:  0.25
    ear  poly  label  prediction
0  0.63  0.62   1.00        1.00
1  0.63  0.62   1.00        0.75
2  0.20  0.25   0.50        0.50
3  0.11  0.05   0.25        0.50
4  0.00  0.00   0.25        0.50
5  0.02  0.06   0.25        0.50
6  0.64  0.56   1.00        1.00
CLASSIFICATION:  0.5
======= BLINK NUMBER 3: =======
LABEL:  0.25
    ear  poly  label  prediction
0  0.76  0.67   0.75        1.00
1  0.34  0.26   0.50        0.50
2  0.17  0.11   0.25        0.50
3  0.26  0.20   0.50        0.50
4  0.21  0.15   0.50        0.75
5  0.41  0.37   0.75        0.75
6  0.75  0.60   1.00        1.00
CLASSIFICATION:  0.5
